In [2]:
# !pip install wimbd

In [22]:
import pandas as pd
df = pd.read_pickle('./data.output.gen_poems.v3.pkl').reset_index()
df = df[df._model.str.contains('olmo2')]
ids = df._id.value_counts()
ids = ids[ids>=13].index.tolist()
len(ids), ids[:3]

(4729,
 ['english/keachben/Z200407698',
  'c20-american/am22020/Z200229960',
  'english/william5/Z300537925'])

In [70]:
import os
from wimbd.es import es_init, count_documents_containing_phrases
es = es_init(os.path.expanduser('~/Downloads/es_config_dolma_1_7_2.yml'))
from hashstash import HashStash
stash = HashStash('find_in_dolma', engine='pairtree', compress=False, b64=False)
# stash.clear()
df_meta=pd.read_csv(os.path.expanduser('~/lltk_data/corpora/chadwyck_poetry/metadata.csv')).set_index('id')


/var/folders/d9/tm2sn8_12xsc6p6nfgtrd5h40000gp/T/ipykernel_50206/3973087862.py:7: DtypeWarning: Columns (12,15,25,31,33,37,40,45,46,49,51,56,57,59,60,69,72,73,78,84,86,95,97,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta=pd.read_csv(os.path.expanduser('~/lltk_data/corpora/chadwyck_poetry/metadata.csv')).set_index('id')


In [84]:
print('\n'.join(df[df._id=="english/brownieb/Z200289174"].line_real))
# df_meta.loc['c20-american/am22041/Z300232794'].dropna()

What can I give thee back, O liberal
And princely giver, who hast brought the gold
And purple of thine heart, unstained, untold,
And laid them on the outside of the wall
For such as I to take or leave withal,
In unexpected largess? am I cold,
Ungrateful, that for these most manifold
High gifts, I render nothing back at all?
Not so; not cold, -- but very poor instead.
Ask God who knows. For frequent tears have run
The colours from my life, and left so dead
And pale a stuff, it were not fitly done
To give the same as pillow to thy head.
Go farther! let it serve to trample on.


In [53]:
@stash.stashed_result
def find_in_dolma(id, index='docs_v1.7_2024-06-04'):
    dfg = df.query(f'_id=="{id}"')
    n = dfg.iloc[0]._first_n_lines
    dfg = dfg[dfg.line_num<=n]
    
    query = [ln.strip() for ln in dfg.sort_values('line_num').line_real]
    res = count_documents_containing_phrases(index, query, es=es, all_phrases=True)
    return {
        'id':id,
        'lines':query,
        'count':res
    }

In [55]:
find_in_dolma('english/keachben/Z200407698')

{'id': 'english/keachben/Z200407698',
 'lines': ['Ye Saints remember God always,',
  'remember he is nigh;',
  'Nay, with us all in every place,',
  'and on us sets his Eye.',
  'O Lord, out of thy piercing Sight'],
 'count': 0}

In [96]:
ddf=find_in_dolma.stash.df.sort_values('count',ascending=False)
ddf['found']=ddf['count']>0
ddf[ddf['count']>0]
print(ddf.found.mean()*100)
ddf.head(25)

8.691055191372383


,id,lines,count,found
2913,c20-american/am20114/Z300221220,"[Tambourines!, Tambourines!, Tambourines, To t...",219,True
4458,english/wattsisa/Z400522989,"[When I survey the wondrous cross, On which th...",99,True
2172,english-ed2/miscell3/Z300440750,"[Go, lovely Rose!, Tell her, that wastes her t...",97,True
1016,c20-american/am20114/Z300220672,"[I must say, Yes, sir,, To you all the time., ...",60,True
2824,english/wattsisa/Z400522722,"[Come let us join our cheerful songs, With ang...",31,True
3242,english/wattsisa/Z400522825,"[Welcome, sweet day of rest,, That saw the Lor...",23,True
503,english/browniro/Z200290194,[At the midnight in the silence of the sleep-t...,16,True
3484,c20-english/yeats/Z500351458,"['Put off that mask of burning gold, With emer...",16,True
835,c20-english/ep20142/Z200584677,"[Breathless, we flung us on the windy hill,, L...",11,True
81,english/smartchr/Z200487468,"[Jesus Christ is risen to day,, Hallelujah., O...",10,True


In [95]:
from tqdm.auto import tqdm
import random
out = []
random.shuffle(ids)
iterr=tqdm(ids)
for id in iterr:
    try:
        res = find_in_dolma(id)
        iterr.set_description(f'found {res["count"]} matches for {res["id"]}: {res["lines"][0]}')
        out.append(res)
    except Exception:
        pass
    # break
odf = pd.DataFrame(out)
odf

  0%|          | 0/4729 [00:00<?, ?it/s]

,id,lines,count
0,english/somervil/Z200490119,"[Tho' close immured, poor Captive Maid!, Young...",0
1,english/jordanth/Z300406478,"[Elizabeth inspire me, then I shall, Write nou...",0
2,english/elliotte/Z300346468,[Though thou no cloud in breezeless heaven beh...,0
3,african-american/rayhenri/Z200473045,"[Far, far out lie the white sails all at rest;...",0
4,c20-english/ep30092/Z300604166,"[The bush at the end of the road, is earmarked...",0
...,...,...,...
4724,c20-english/ep30113/Z400605743,"[Whatever is beyond my reach, I shall not reac...",0
4725,c20-american/am22097/Z200236403,"[Migrating along (butterfly-like,, actually) I...",0
4726,faber/fa20502/Z400564935,"[Sea's mass is clumsy,, Wrestling with hills a...",0
4727,modern/car1001/Z200547693,"[They posted us to battle camp., Three years s...",0
